<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/C%C3%B3digo_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(ggplot2)
#install.packages('dplyr')
library(dplyr)
#install.packages('MASS')
library(MASS)
#install.packages('randomForest')
library(randomForest)
#install.packages('caret')
library("caret")
library("googledrive")
#install.packages("mice")
library(mice)


In [7]:
id <- "1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE" # google file ID
df <- read.csv(sprintf("https://docs.google.com/uc?id=%s&export=download", id))
head(df)

,X,vigente,vigenteAvg3M,vencido,vencidoAvg3M,avgcontingenciaU1M,avgcontingenciaU3M,sumcontingenciaU1M,sumcontingenciaU3M,maxcontingenciaU1M,⋯,numerobancosprivados,numerobancospublicos,numerofinancieras,numerobancos,numerode5ult18meses,numerode2Bult18meses,numerode3ult18meses,edad,numeromesesreportados,ingreso
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,1,4282.155,4679.336,0.000,0.000,4923.124,3918.527,19692.50,18439.94,6653.07,⋯,1,0,3,1,0,0,0,63,24,31888.47
2,2,9793.123,9450.927,83970.243,91126.581,20809.053,19126.816,41618.11,49280.50,30881.25,⋯,0,1,1,1,4,0,2,67,24,62341.14
3,3,531419.702,512057.085,0.000,0.000,36813.149,46144.419,36813.15,36789.29,36813.15,⋯,2,1,0,3,0,0,0,NA,24,100599.76
4,4,0.000,0.000,2266.729,2633.112,11533.815,10488.339,23067.63,24025.95,19044.72,⋯,4,0,0,4,4,2,3,NA,24,46102.90
5,5,112394.057,110856.021,0.000,0.000,41131.334,42820.024,82262.67,77614.23,43944.58,⋯,4,0,0,4,0,0,0,65,14,109419.22
6,6,0.000,0.000,15136.639,15028.350,38851.830,36968.398,38851.83,37783.35,38851.83,⋯,0,0,3,0,9,0,1,NA,24,59375.89


In [ ]:
str(df)

In [9]:
#Convertir variables cualitativas

#unique(df$peorcalificacionBCU)
df$peorcalificacionBCU = ifelse(df$peorcalificacionBCU == '1C', 1,
ifelse(df$peorcalificacionBCU == '2A',2, ifelse(df$peorcalificacionBCU == '2B',3,
ifelse(df$peorcalificacionBCU == '3',4, ifelse(df$peorcalificacionBCU == '4',5,6)))))

unique(df$peorcalificacionBCU)

[1] 1 4 6 3 5 2

In [ ]:
# Transformación de las variables categóricas en factor

df <- transform(df, peorcalificacionBCU = as.factor(peorcalificacionBCU))

str(df)

In [ ]:
# Evaluando Missing Values

na_count <-sapply(df, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count

In [ ]:
# Completando Missing Values

mice_imputes = mice(df, m=5, maxit = 50)
#methods(mice)

df = complete(mice_imputes,5)

na_count <-sapply(df, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count

In [ ]:
head(df)

In [37]:
# Partición del dataset entre train (80%) y test (20%)

set.seed(27)
df_index = createDataPartition(df$ingreso, p=0.80, list = FALSE)
df_train <- df[df_index, ]
df_test <- df[-df_index, ]

In [ ]:
# Entrenamiento del modelo

rf = randomForest(ingreso ~ ., data = df_train, mtry = 3, importance = TRUE, ntree = 500)

rf_pred = predict(rf, newdata = df_test[,-25], type="response")

#rf_tabla = table(ifelse(rf_pred[,2]>0.5,1,0), df_test[,25])
#rf_tabla
#rf_er = 1- sum(diag(rf_tabla))/sum(rf_tabla)
#rf_er
#rf_er = rf_er*100

In [ ]:
# Optimización del modelo Random Forest

set.seed(27)

k_folds = 5
folds = caret::createFolds(c(1:nrow(df)),k=k_folds)
m = seq(1,10, by=2)
narbol = seq(100, 1000, by = 100)
resultadosrf = data.frame()

set.seed(27)

for (k in 1:k_folds){
  for (j in m){
    for (i in narbol){
      train = df[-folds[[k]],]
      test = df[folds[[k]],]
    rf_opt = randomForest(ingreso ~ ., data = train, mtry = j, importance = TRUE, ntree = i)
    rf_pred_opt = predict(rf_opt, newdata = test[,-1], type="response")
    #rf_tabla_opt = table(ifelse(rf_pred_opt[,2]>f,1,0), test[,1])
    #rf_er_opt = 1- sum(diag(rf_tabla_opt))/sum(rf_tabla_opt)
    result = c("k_folds"=k, "m"=j, "narbol"=i, "error"= rf_er_opt)
    resultadosrf = rbind(resultadosrf, result)
  }
  }
}


resultadosrf <- setNames(resultadosrf,c("k_folds","m","narbol","error"))
resultadosrf_cv <- aggregate(resultadosrf[,4],list(resultadosrf$m,resultadosrf$narbol),mean)
resultadosrf_cv <- setNames(resultadosrf_cv,c("m","narbol","error"))

#View(resultadosrf_cv)

resultadosrf_cv_ord <- resultadosrf_cv[order(resultadosrf_cv$error),]
head(resultadosrf_cv_ord)

In [ ]:
importance(rf)
varImpPlot(rf)

In [ ]:
# Predicción con el modelo optimizado

id <- "1dUb4neNxfSpCGRGucf_A-kcIV5t6QL1gSXDPi4uL2HA" 
df_val <- read.csv(sprintf("https://docs.google.com/uc?id=%s&export=download", id))

df_val$peorcalificacionBCU = ifelse(df_val$peorcalificacionBCU == '1C', 1,
ifelse(df_val$peorcalificacionBCU == '2A',2, ifelse(df_val$peorcalificacionBCU == '2B',3,
ifelse(df_val$peorcalificacionBCU == '3',4, ifelse(df_val$peorcalificacionBCU == '4',5,6)))))

df_val <- transform(df_val, peorcalificacionBCU = as.factor(peorcalificacionBCU))


rf_pred_def = predict(rf_final, newdata = df_val, type="response")
#View(rf_pred_def)

#df_def = data.frame(ifelse(rf_pred_def[,2]>corte,1,0))
df_def <- setNames(df_def, "prediccion")
#View(df_def)

write.csv(x = df_def, file = "df_def.csv", row.names = TRUE) 